In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from keras_vggface.vggface import VGGFace

# Uwaga: Może być konieczne zainstalowanie tej biblioteki w terminalu JupyterLab:
# pip install keras-vggface
# pip install keras_applications

def create_base_model(num_classes, input_shape=(224, 224, 3)):
    """
    Tworzy model bazowy VGGFace (ResNet-50) gotowy do fine-tuningu.

    Argumenty:
    num_classes (int): Liczba klas wyjściowych (liczba tożsamości w Twoim zbiorze).
    input_shape (tuple): Rozmiar obrazów wejściowych.

    Zwraca:
    tf.keras.models.Model: Skompilowany model Keras gotowy do treningu.
    """
    
    # 1. Załaduj model bazowy VGGFace (np. architekturę ResNet-50)
    # include_top=False oznacza, że nie ładujemy oryginalnych warstw klasyfikacyjnych.
    # To jest kluczowe dla fine-tuningu.
    base_model = VGGFace(model='resnet50', 
                         include_top=False, 
                         input_shape=input_shape, 
                         pooling='avg') # 'avg' od razu spłaszcza wyjście

    # 2. "Zamrożenie" warstw bazowych
    # Nie chcemy na nowo trenować milionów parametrów, które już nauczyły się
    # rozpoznawać ogólne cechy twarzy. Ustawiamy je jako nietrenowalne.
    print(f"Liczba warstw w modelu bazowym: {len(base_model.layers)}")
    for layer in base_model.layers:
        layer.trainable = False
        
    # 3. Dodanie nowych warstw (tzw. "głowy" klasyfikacyjnej)
    # To jest "miejsce na fine-tuning". Tylko te warstwy będą trenowane od zera
    # na Twoim zbiorze danych z okluzją.
    
    # Pobierz wyjście z zamrożonego modelu
    x = base_model.output
    
    # Możesz tu dodać więcej warstw, jeśli chcesz, np:
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x) # Dropout pomaga w walce z przeuczeniem

    # Ostatnia warstwa - klasyfikator.
    # Musi mieć tyle neuronów, ile masz klas (tożsamości).
    # Używamy 'softmax' do klasyfikacji wieloklasowej.
    predictions = Dense(num_classes, activation='softmax')(x)

    # 4. Stworzenie finalnego modelu
    # Łączymy stary "korpus" (base_model.input) z nową "głową" (predictions)
    model = Model(inputs=base_model.input, outputs=predictions)

    # 5. Kompilacja modelu
    # Musimy zdefiniować optymalizator (jak model ma się uczyć)
    # i funkcję straty (jak mierzymy błąd).
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])

    print("Model gotowy do fine-tuningu.")
    model.summary()
    
    return model

if __name__ == '__main__':
    # Przykład użycia:
    # Załóżmy, że masz 50 różnych osób w swoim zbiorze danych
    NUM_CLASSES = 50 
    
    model = create_base_model(num_classes=NUM_CLASSES)
    
    # Teraz możesz wczytać swoje dane i uruchomić:
    # model.fit(train_generator, validation_data=validation_generator, epochs=10)